# IPython interface for running GPU DSM + fitting on cluster

In [1]:
from dsm_tools import ssh_init, job_start, job_done, job_transfer_clean, uuid, os, server, username, password, Calculation, CalculationStar, CalculationLinear, IterationChain
from fdt_fit import fdt_fit, fdt, fdtvec
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
def mix_spectra(w, x, y):
    xWeight = w #Fraction of linear
    yWeight = 1 - w

    #Calculate p_eq of the mix
    mix_lambdaArr = np.append(x.lambdaArr, y.lambdaArr)
    mix_gArr = np.append(np.multiply(x.gArr, x.lambdaArr)/np.dot(x.gArr, x.lambdaArr) * xWeight, np.multiply(y.gArr, y.lambdaArr)/np.dot(y.gArr, y.lambdaArr) * yWeight)

    #Sort by relaxation times tau
    sort = np.argsort(mix_lambdaArr)
    mix_lambdaArr = mix_lambdaArr[sort]
    mix_gArr = mix_gArr[sort]

    return mix_lambdaArr, np.divide(mix_gArr,mix_lambdaArr)/np.sum(np.divide(mix_gArr,mix_lambdaArr))

In [3]:
nChains = 5000
CDtoggle = 0
simTime = 1000000
num_gpu = 4

#Linear chain
nk = 50

#Star chain
nArms = 3
nkArms = [6, 6, 6]

In [4]:
x, y = CalculationLinear('x', nk, 0, nChains, num_gpu, simTime), CalculationStar('y', nArms, nkArms, 0, nChains, num_gpu, simTime)

In [7]:
weights = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0] #weight fraction of linear chains in star-linear blend
iterations = map(lambda x: IterationChain(x),weights)

#Create iteration chains for different blends
iterations[0].add((CalculationLinear(nk, 1, nChains, num_gpu, simTime)))
[i.add((CalculationLinear(nk, 1, nChains, num_gpu, simTime), CalculationStar(nArms, nkArms, 1, nChains, num_gpu, simTime))) for i in iterations[1:-1]]
iterations[-1].add((CalculationStar(nArms, nkArms, 1, nChains, num_gpu, simTime)))

To start a claculation of a **linear chain**, call `CalculationLinear(Nc, CDtoggle, Nchains, Ngpus, simTime)`

To start a claculation of a **star-branched chain**, call `CalculationStar(nArms, NcArms, CDtoggle, Nchains, Ngpus, simTime)`

### Zeroth iteration (pure linear, pure star; no constraint dynamics)

In [8]:
x.calc()
y.calc()

In [10]:
x.check_self()
y.check_self()

In [11]:
x.plot_fit_results()

In [30]:
save(x, 'x.pkl')

In [12]:
y.plot_fit_results()

### Longest relaxation time for a pure linear and a pure star

In [14]:
#Calculate p_cr for different blends
iterations[0].chain[-1].set_pcd_cr_input(x.lambdaArr,x.gArr)

for i in iterations[1:-1]:
    a,b = mix_spectra(i.w, x, y)
    i.chain[-1][0].set_pcd_cr_input(a,b)
    i.chain[-1][1].set_pcd_cr_input(a,b)

iterations[-1].chain[-1].set_pcd_cr_input(y.lambdaArr,y.gArr)

In [15]:
#pure linear
iterations[0].chain[-1].calc()

In [31]:
iterations[0].chain[-1].check_self()

In [41]:
print 'Pure linear'
iterations[0].chain[-1].plot_fit_results()

In [17]:
#pure star
iterations[-1].chain[-1].calc()

In [33]:
iterations[-1].chain[-1].check_self()

In [42]:
print 'Pure star'
iterations[-1].chain[-1].plot_fit_results()

In [20]:
iterations[1].chain[-1][0].calc()
iterations[1].chain[-1][1].calc()

In [35]:
iterations[1].chain[-1][0].check_self()
iterations[1].chain[-1][1].check_self()

In [39]:
print 'Linear (90% linear + 10% star)'
iterations[1].chain[-1][0].plot_fit_results()

In [140]:
print 'Star (90% linear + 10% star)'
iterations[1].chain[-1][1].plot_fit_results()

In [22]:
iterations[2].chain[-1][0].calc()
iterations[2].chain[-1][1].calc()

In [141]:
iterations[2].chain[-1][0].check_self()
iterations[2].chain[-1][1].check_self()

In [142]:
print 'Linear (80% linear + 20% star)'
iterations[2].chain[-1][0].plot_fit_results()

In [44]:
print 'Star (80% linear + 20% star)'
iterations[2].chain[-1][1].plot_fit_results()

In [24]:
iterations[3].chain[-1][0].calc()
iterations[3].chain[-1][1].calc()

In [46]:
iterations[3].chain[-1][0].check_self()
iterations[3].chain[-1][1].check_self()

In [48]:
print 'Linear (70% linear + 30% star)'
iterations[3].chain[-1][0].plot_fit_results()

In [47]:
print 'Star (70% linear + 30% star)'
iterations[3].chain[-1][1].plot_fit_results()

In [26]:
iterations[4].chain[-1][0].calc()
iterations[4].chain[-1][1].calc()

In [49]:
iterations[4].chain[-1][0].check_self()
iterations[4].chain[-1][1].check_self()

In [51]:
print 'Linear (60% linear + 40% star)'
iterations[4].chain[-1][0].plot_fit_results()

In [50]:
print 'Star (60% linear + 40% star)'
iterations[4].chain[-1][1].plot_fit_results()

In [90]:
iterations[4].chain[-1][1].plot_fit_results()

In [28]:
iterations[5].chain[-1][0].calc()
iterations[5].chain[-1][1].calc()

In [125]:
iterations[5].chain[-1][0].check_self()
iterations[5].chain[-1][1].check_self()

In [54]:
print 'Linear (50% linear + 50% star)'
iterations[5].chain[-1][0].plot_fit_results()

In [126]:
print 'Star (50% linear + 50% star)'
iterations[5].chain[-1][1].plot_fit_results()

In [56]:
iterations[6].chain[-1][0].calc()
iterations[6].chain[-1][1].calc()

In [127]:
iterations[6].chain[-1][0].check_self()
iterations[6].chain[-1][1].check_self()

In [103]:
print 'Linear (40% linear + 60% star)'
iterations[6].chain[-1][0].plot_fit_results()

In [105]:
print 'Star (40% linear + 60% star)'
iterations[6].chain[-1][1].plot_fit_results()

In [58]:
iterations[7].chain[-1][0].calc()
iterations[7].chain[-1][1].calc()

In [128]:
iterations[7].chain[-1][0].check_self()
iterations[7].chain[-1][1].check_self()

In [108]:
print 'Linear (30% linear + 70% star)'
iterations[7].chain[-1][0].plot_fit_results()

In [107]:
print 'Star (30% linear + 70% star)'
iterations[7].chain[-1][1].plot_fit_results()

In [60]:
iterations[8].chain[-1][0].calc()
iterations[8].chain[-1][1].calc()

In [129]:
iterations[8].chain[-1][0].check_self()
iterations[8].chain[-1][1].check_self()

In [130]:
iterations[8].chain[-1][0].plot_fit_results()

In [70]:
iterations[8].chain[-1][1].plot_fit_results()

In [62]:
iterations[9].chain[-1][0].calc()
iterations[9].chain[-1][1].calc()

In [131]:
iterations[9].chain[-1][0].check_self()
iterations[9].chain[-1][1].check_self()

In [132]:
iterations[9].chain[-1][0].plot_fit_results()

In [72]:
iterations[9].chain[-1][1].plot_fit_results()

### Look at the longest relaxation time of star-branched

In [143]:
iterations[1].chain[-1][1].lambdaArr

In [144]:
iterations[2].chain[-1][1].lambdaArr

In [145]:
iterations[3].chain[-1][1].lambdaArr

In [146]:
iterations[4].chain[-1][1].lambdaArr

In [147]:
iterations[5].chain[-1][1].lambdaArr

In [148]:
iterations[6].chain[-1][1].lambdaArr

In [149]:
iterations[7].chain[-1][1].lambdaArr

In [150]:
iterations[8].chain[-1][1].lambdaArr

In [151]:
iterations[9].chain[-1][1].lambdaArr

In [152]:
iterations[10].chain[-1].lambdaArr

### Look at the longest relaxation time of linear

In [153]:
iterations[0].chain[-1].lambdaArr

In [154]:
iterations[1].chain[-1][0].lambdaArr

In [155]:
iterations[2].chain[-1][0].lambdaArr

In [156]:
iterations[3].chain[-1][0].lambdaArr

In [157]:
iterations[4].chain[-1][0].lambdaArr

In [158]:
iterations[5].chain[-1][0].lambdaArr

In [159]:
iterations[6].chain[-1][0].lambdaArr

In [160]:
iterations[7].chain[-1][0].lambdaArr

In [161]:
iterations[8].chain[-1][0].lambdaArr

In [162]:
iterations[9].chain[-1][0].lambdaArr

### Convergence of pure linear

In [123]:
x.token

In [124]:
iterations[0].chain[-1].token

In [111]:
iterations[0].chain[-1].lambdaArr

In [89]:
x.lambdaArr

In [1]:
x.fdt_result_x

In [91]:
x.plot_fit_results()

In [92]:
iterations[0].chain[-1].plot_fit_results()

### Convergence of pure star

In [137]:
y.lambdaArr

In [135]:
iterations[-1].chain[-1].lambdaArr

In [138]:
y.token

In [139]:
iterations[-1].chain[-1].token